##### Nama    : Dafa Ghani Abdul Rabbani 
##### NPM     : 140810230022 
##### Kelas   : B 
##### -------------------------------------------------------------------------------------------------------------------------------------------

#### Import library

In [1]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

#### Ambil dataset foto dari URL

In [2]:
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
local_file = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/horse-or-human')

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
local_file = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

#### Fungsi untuk membuat model

In [3]:
def solution_05():
    TRAINING_DIR = 'data/horse-or-human'
    VALIDATION_DIR = 'data/validation-horse-or-human'
    
    train_datagen = ImageDataGenerator(
        rescale=1/255,
        rotation_range=40,
        horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)
    
    # Mempersiapkan generator untuk training data
    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )
    
    # Mempersiapkan generator untuk validation data
    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
    )

    # Membangun model CNN
    model = tf.keras.models.Sequential([
        # Input layer
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        # Layer kedua
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        # Layer ketiga
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        # Flatten layer
        tf.keras.layers.Flatten(),
        
        # Dense layers
        tf.keras.layers.Dense(128, activation='relu'), 
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer=RMSprop(learning_rate=0.0001),
        metrics=['accuracy']
    )
    
    # Melatih model
    history = model.fit(
        train_generator,
        steps_per_epoch=25,  # 800 images / 32 batch size
        epochs=15,
        validation_data=validation_generator,
        validation_steps=8,  # ~256 validation images / 32 batch size
        verbose=1
    )
    
    return model

#### Hasil Train dan Test Model

In [4]:
model = solution_05()

TRAINING_DIR = 'data/horse-or-human'
VALIDATION_DIR = 'data/validation-horse-or-human'

# Data augmentation untuk training
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

# Hanya rescaling untuk validasi
validation_datagen = ImageDataGenerator(rescale=1./255)

# Generator untuk training data
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Generator untuk validation data
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=25,  # ~800 images / 32 batch size
    epochs=15,
    validation_data=validation_generator,
    validation_steps=8,  # ~256 validation images / 32 batch size
    verbose=1,
    callbacks=[early_stopping]
)

# Hasil final dari proses training
print("\n=== HASIL FINAL DARI PROSES TRAINING ===")
print(f"Training Accuracy (Epoch Terakhir): {history.history['accuracy'][-1]:.4f}")
print(f"Validation Accuracy (Epoch Terakhir): {history.history['val_accuracy'][-1]:.4f}")

# Evaluasi model pada data validasi
print("\n=== HASIL EVALUASI PADA DATA VALIDASI ===")
evaluation = model.evaluate(validation_generator, verbose=1)
print(f"Evaluation Loss: {evaluation[0]:.4f}")
print(f"Evaluation Accuracy: {evaluation[1]:.4f}")

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


c:\Users\Dafa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Dafa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5651 - loss: 1.5536 - val_accuracy: 0.6992 - val_loss: 0.6472
Epoch 2/15
 8/25 ━━━━━━━━━━━━━━━━━━━━ 8s 517ms/step - accuracy: 0.8863 - loss: 0.3205

c:\Users\Dafa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 180ms/step - accuracy: 0.8786 - loss: 0.3434 - val_accuracy: 0.7383 - val_loss: 0.6276
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 579ms/step - accuracy: 0.8254 - loss: 0.3366 - val_accuracy: 0.6953 - val_loss: 0.5847
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.8945 - loss: 0.2483 - val_accuracy: 0.7148 - val_loss: 0.5722
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 459ms/step - accuracy: 0.9070 - loss: 0.2241 - val_accuracy: 0.5703 - val_loss: 0.6180
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.9201 - loss: 0.1804 - val_accuracy: 0.6172 - val_loss: 0.5718
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 452ms/step - accuracy: 0.9243 - loss: 0.1652 - val_accuracy: 0.6914 - val_loss: 0.5374
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 142ms/step - accuracy: 0.9706 - loss: 0.1067 - val_accuracy: 0.6875 - val_loss: 0.5421
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 435ms/step - accuracy: 0.9465 - loss: 0.1373 - val_accuracy: 0.6602 - 

In [ ]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_05()
    model.save("model_05.h5")